In [1]:
import pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

In [2]:
file_name = 'YutaMouse41-150903.nwb'

In [3]:
nwb_io = NWBHDF5IO(file_name, mode='r')
nwb = nwb_io.read()

In [7]:
nwb2widget(nwb)

## Add custom tab

In [8]:
import copy as cp

from nwbwidgets.view import default_neurodata_vis_spec
from nwbwidgets.misc import show_annotations

vis_spec = cp.deepcopy(default_neurodata_vis_spec)

vis_spec[pynwb.misc.AnnotationSeries]['star'] = lambda x: show_annotations(x, marker='*')

In [9]:
nwb2widget(nwb, vis_spec)